This jupyter notebook can be run on a x86 CPU (Intel or AMD) and ARM CPU (Apple Mx), with or without a Nvidia GPU. We recommend running it on a Linux machine.

In [7]:
# Set the path to the dataset.
# Here we download and use a dataset containing 354 images showing clothes.
# If you want to use your own dataset, just set the dataset_path to it.
dataset_path = "dataset_clothing_images"
!git clone https://github.com/lightly-ai/dataset_clothing_images.git {dataset_path}

Cloning into './dataset_clothing_images'...
Receiving objects: 100% (376/376), 9.55 MiB | 44.65 MiB/s, done.


In [6]:
# Install the Lightly Python Client
!pip3 install lightly

# Install the Lightly Worker and do a quick sanity check
# If these commands fail, follow our docker installation guide at https://docs.lightly.ai/docs/install-lightly#docker
!docker pull lightly/worker:latest
!docker run --shm-size="1024m" --rm -it lightly/worker:latest sanity_check=True

latest: Pulling from lightly/worker
Status: Image is up to date for lightly/worker:latest
[2024-03-26 12:27:06] Lightly Worker Solution v2.11.1
[2024-03-26 12:27:06] Congratulations! It looks like the Lightly container is running!


In [9]:
# Schedule a run
from lightly.api import ApiWorkflowClient
from lightly.openapi_generated.swagger_client import DatasetType, DatasourcePurpose

lightly_token = "CHANGE_ME" # Copy the API_TOKEN from https://app.lightly.ai/preferences

# Create the Lightly client to connect to the API.
client = ApiWorkflowClient(token=lightly_token)

# Create the dataset on the Lightly Platform.
# If a dataset with this name already exists, you have to change the name.
client.create_dataset(
    dataset_name="clothing-small",
    dataset_type=DatasetType.IMAGES,
)

# Configure the datasource.
client.set_local_config(
    purpose=DatasourcePurpose.INPUT,
)
client.set_local_config(
    purpose=DatasourcePurpose.LIGHTLY,
)

# Schedule a run on the dataset to select 50 samples.
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={"shutdown_when_job_finished": True},
    selection_config={
        "n_samples": 50,
        "strategies": [
            {"input": {"type": "EMBEDDINGS"}, "strategy": {"type": "DIVERSITY"}}
        ],
    },
)

In [10]:
# Run the Lightly Worker to process the scheduled run.
# If you have a Nvidia GPU, add the flag to use it: --gpus all

from pathlib import Path
absolute_dataset_path = Path(dataset_path).absolute()
absolute_lightly_path = Path("lightly").absolute()

!docker run --shm-size="1024m" --rm -it \
    -v {absolute_dataset_path}:/input_mount:ro \
    -v {absolute_lightly_path}:/lightly_mount \
    -e LIGHTLY_TOKEN={lightly_token} \
    lightly/worker:latest

[2024-03-26 12:27:54] Lightly Worker Solution v2.11.1
[2024-03-26 12:27:55] You are using docker build: Tue Mar 12 07:56:29 UTC 2024.
[2024-03-26 12:27:55] Starting worker with id '65806b455ca68c93b29ad6b3'...
[2024-03-26 12:27:55] Worker 2.11.1 can only process jobs scheduled with Lightly Python client 1.5 or higher.
[2024-03-26 12:27:55] Worker with labels '[]' started. Waiting for jobs...
[2024-03-26 12:27:56] Found 1 open jobs.
[2024-03-26 12:27:57] Started job with job_id '6602bf3ef5e7d0833c9264e8'.
...
...
[2024-03-26 12:29:21] Done!
[2024-03-26 12:29:24] Finished compute worker run successfully.
[2024-03-26 12:29:24] Shutting down...


Congratulations! You succesfully ran the Lightly solution.
Now you can view and explore the dataset interactively on the [Lightly Platform](https://app.lightly.ai).
To not only see the metadata and distribution, but also the images itself, you need to serve them from your local disk to your local browser by using the `lightly-serve` CLI command:

In [11]:
!lightly-serve input_mount={absolute_dataset_path} lightly_mount={absolute_lightly_path}

Starting server, listening at 'localhost:3456'
Serving files in '/GitHub/lightly-solution-all-in-one-notebook/dataset_clothing_images' and '/GitHub/lightly-solution-all-in-one-notebook/lightly'


In case your browser runs on a different machine than your notebook, you also need to forward a port, see our [docs](https://docs.lightly.ai/docs/local-storage#view-local-data-in-remote-machine-in-lightly-platform).